In [1]:
# Use wfdb to access the PhysioNet Data
import wfdb
import wfdb.processing
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np

# Use the signal for filters
from scipy.signal import butter, filtfilt

In [16]:
# Load both the header (.hea) and the signal data (.mat) using wfdb
record = wfdb.rdrecord('0610_012_025_ECG')

In [17]:
# Print the contents of the header file
print("Record Name:", record.record_name)
print("Sampling Frequency:", record.fs)
print("Signal Length (samples):", record.sig_len)
print("Number of Signals (channels):", record.n_sig)
print("Signal Names:", record.sig_name)
print("Signal Units:", record.units)

# Access the signal data (numpy array)
ecg_signal_data = record.p_signal.flatten()  # Assuming one channel and flatten to 1D

# Check the shape of the signal data
print("Signal Data Shape:", ecg_signal_data.shape)


Record Name: 0610_012_025_ECG
Sampling Frequency: 250
Signal Length (samples): 900000
Number of Signals (channels): 1
Signal Names: ['ECG']
Signal Units: ['nu']
Signal Data Shape: (900000,)


In [18]:
# High-pass filter to remove baseline drift (cutoff frequency = 0.5 Hz)
b_high, a_high = butter(1, 0.5 / (250 / 2), btype='high')
filtered_signal = filtfilt(b_high, a_high, ecg_signal_data)

# Low-pass filter to remove high-frequency noise (cutoff frequency = 50 Hz)
b_low, a_low = butter(1, 50 / (250 / 2), btype='low')
filtered_signal = filtfilt(b_low, a_low, filtered_signal)

In [19]:
r_peaks_indices = wfdb.processing.gqrs_detect(filtered_signal, fs=250)

In [20]:
rr_intervals = np.diff(r_peaks_indices) / 250  # Convert RR intervals to seconds
heart_rate = 60 / rr_intervals  # Convert to beats per minute
average_heart_rate = np.mean(heart_rate)

In [21]:
heart_rate

array([142.85714286, 131.57894737, 197.36842105, ..., 205.47945205,
       118.11023622, 205.47945205])

In [22]:
# Create the Plotly plot for the ECG Signal
trace_signal = go.Scatter(
    y=filtered_signal,
    mode='lines',
    name='ECG Signal'
)

# Create the Plotly plot for the R-Peaks
r_peaks_y = [filtered_signal[i] for i in r_peaks_indices]  # Get the amplitudes of the R-peaks
trace_r_peaks = go.Scatter(
    x=r_peaks_indices,
    y=r_peaks_y,
    mode='markers',
    name='R-Peaks',
    marker=dict(color='red', size=8, symbol='x')  # Customize the appearance
)

# Define the layout
layout = go.Layout(
    title='Scrollable ECG Signal with R-Peaks',
    xaxis=dict(
        title='Time (samples)',
        rangeslider=dict(visible=True),  # Add a range slider to make it scrollable
    ),
    yaxis=dict(
        title='Amplitude',
    ),
    width=1200,  # Set a wider plot
    height=500
)

# Create the figure and add both traces
fig = go.Figure(data=[trace_signal, trace_r_peaks], layout=layout)

# Show the plot in an interactive window
pyo.plot(fig, filename='scrollable_ecg_with_r_peaks.html')

'scrollable_ecg_with_r_peaks.html'